In [4]:
import os.path
import json
import datetime

from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.auth.transport.requests import Request

In [8]:
SCOPES = ["https://www.googleapis.com/auth/calendar.readonly"]

def get_credentials():
    """Retrieve stored credentials from tokens.json."""
    if os.path.exists("tokens.json"):
        with open("tokens.json", "r") as token_file:
            tokens = json.load(token_file)
            return [Credentials.from_authorized_user_info(json.loads(token), SCOPES) for token in tokens]
    return []

def save_credentials(creds):
    """Save credentials to tokens.json."""
    tokens = []
    if os.path.exists("tokens.json"):
        with open("tokens.json", "r") as token_file:
            tokens = json.load(token_file)
    tokens.append(creds.to_json())
    with open("tokens.json", "w") as token_file:
        json.dump(tokens, token_file)

def main():
    """Shows basic usage of the Google Calendar API."""
    creds_list = get_credentials()
    if not creds_list:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
        save_credentials(creds)
        creds_list = [creds]
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
        creds = flow.run_local_server(port=0, authorization_prompt_message="Select the account to use or add a new one: ")
        save_credentials(creds)
        creds_list.append(creds)

    start_date = datetime.datetime(2023, 3, 15, 0, 0, 0).isoformat() + "Z"
    end_date = datetime.datetime(2024, 3, 16, 23, 59, 59).isoformat() + "Z"

    try:
        for creds in creds_list:
            service = build("calendar", "v3", credentials=creds)

            events_result = (
                service.events()
                .list(
                    calendarId="primary",
                    timeMin=start_date,
                    timeMax=end_date,
                    singleEvents=True,
                    orderBy="startTime",
                )
                .execute()
            )
            events = events_result.get("items", [])

            print("Events:")
            if not events:
                print("No events found.")
            else:
                for event in events:
                    start = event["start"].get("dateTime", event["start"].get("date"))
                    print(start, event["summary"])

    except Exception as error:
        print(f"An error occurred: {error}")
if __name__ == "__main__":
    main()


Select the account to use or add a new one: 
Events:
2023-05-16 3 mosquetaires
2023-05-17 café linguistique
2023-05-18 Concert mix latino africa
2023-05-19 parc omega
2023-05-20 Zulema
2023-05-21 tam tam
2023-05-21 jardin botanique
2023-05-22 patriotes
2023-05-24 cirque du soleil et photo
2023-05-24 café linguistique
2023-05-25 Québec city
2023-05-28 surf
2023-05-28 tam tam
2023-05-30 Je reviens chez moi
2023-08-05T15:00:00-05:00 Reunión con Evelyn para Gerencia
2023-08-10 Entrega de idea. Gerencia.
2023-08-11 Ejercicio de HPC
2023-08-11 Reunión labSoft
2023-08-12 Reunión con José en la tarde
2023-08-16 1ra entrega LabSoft
2023-08-16T14:00:00-05:00 HPCG1 - S3C1
2023-08-16T20:00:00-05:00 reunión Leo PG 
2023-08-17T20:30:00-05:00 reunión adminSoft
2023-08-18T00:00:00-05:00 enviar plantilla IEEE labsoft 
2023-08-19T09:00:00-05:00 Meeting LabSoft virtual max 
2023-08-22 socialización con el profe de adminSoft
2023-08-23 Entrega de casos de uso y guiones
2023-08-23T19:30:00-05:00 reunión Ad